# Import Libraries

In [1]:
# !pip install jsonlines
# !pip install bertopic

In [1]:
import warnings

# Turn off all user warnings
warnings.filterwarnings("ignore", category=UserWarning)

# Reset warnings filter to its default state (if needed) after your code block
# warnings.filterwarnings("default", category=UserWarning)

In [2]:
import numpy as np
import pandas as pd
import io
import joblib
import os
import time
from tqdm.notebook import tqdm_notebook
from tqdm import tqdm
import pickle

import re, string
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

#nltk.download()
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

from bertopic import BERTopic
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic.representation import MaximalMarginalRelevance

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/joeymeyer/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/joeymeyer/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/joeymeyer/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/joeymeyer/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/joeymeyer/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
2023-11-15 19:02:29.790039: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-15 19:02:29.790071: E tensorflow/c

In [3]:
from wordcloud import WordCloud
from wordcloud import ImageColorGenerator
from wordcloud import STOPWORDS
import matplotlib.pyplot as plt

In [4]:
import jsonlines
import os

## File Access

### Sonia's Machine

In [5]:
# Sonia's Machine
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


2023-11-15 19:02:34.784827: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-15 19:02:34.785025: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-15 19:02:34.809253: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [101]:
%cd ..

/media/joeymeyer/970-evo-plus/Sonia/bertproj


In [6]:
%ls

 bert_filter.ipynb*     combine/   nlp_pipeline.ipynb*   travel_regex.txt*
 bert_training.ipynb*   dolly/    'travel keywords'/     ultrachat/


In [103]:
cwd = os.getcwd()
cwd

'/media/joeymeyer/970-evo-plus/Sonia/bertproj'

### Google Drive

In [ ]:
# from google.colab import files
from google.colab import drive
drive.mount('drive')

In [ ]:
%cd /content/drive/My Drive/'SJSU Classes'/'*2023 Fall DATA298A'/'BERT data'
!ls

In [ ]:
# Define the folder containing the JSONL files
cwd = os.getcwd()

## Import Processed Data - UltraChat

In [13]:
%cd /media/joeymeyer/970-evo-plus/Sonia/bertproj/bertproj/ultrachat/ultrachat_1_output
%ls

/media/joeymeyer/970-evo-plus/Sonia/bertproj/bertproj/ultrachat/ultrachat_1_output
bertopic_model_1.pkl*      topic_results_3.pkl*
bertopic_model_2.pkl*      ultrachat1_travel_df.csv*
processed_data.csv*        ultrachat_heatmap_visualization.png*
processed_with_topic.csv*  ultrachat_hierarchy_visualization.png*
topic_results_1.pkl*       ultrachat_topic_tree.png*
topic_results_2.pkl*       ultrachat_topic_visualization.png*


In [13]:
source = 'ultrachat'
# output_dir = os.path.join(cwd, f'{source}_output')
output_dir = cwd
nr_topics = 100

In [14]:
output_dir

'/media/joeymeyer/970-evo-plus/Sonia/bertproj/ultrachat/ultrachat_rest_output'

In [15]:
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [14]:
df = pd.read_csv('processed_data.csv')
df.head()

,data,combined,cause_clean,bert_summary
0,['How can cross training benefit groups like r...,How can cross training benefit groups like run...,how can cross training benefit group like runn...,improve from be your 5 help the can and 4 like...
1,['Are there any particular physical benefits t...,Are there any particular physical benefits to ...,be there any particular physical benefit to mi...,stay steady more or walk determination managem...
2,"[""What percentage of the Earth's surface is co...",What percentage of the Earth's surface is cove...,what percentage of the earth s surface be cove...,be ocean hidden earth be place role ocean and ...
3,['How does language translation technology imp...,How does language translation technology impac...,how do language translation technology impact ...,part barrier collaboration have software marke...
4,['What is the most popular smartphone brand th...,What is the most popular smartphone brand thes...,what be the most popular smartphone brand thes...,or iphones a capture feature say model and be ...


In [ ]:
# # Assuming 'df' is your DataFrame
# df.dropna(subset=['bert_summary'], inplace=True)

In [ ]:
# save = df
# # df = save

In [17]:
df.shape

(1169743, 6)

In [ ]:
# from sklearn.model_selection import train_test_split

# # Split the data into a training set and a testing set
# df1, df2 = train_test_split(df, test_size=0.5, random_state=42)

# print(df1.shape)
# print(df2.shape)

In [ ]:
# df = df1
# df = df2

## Import Processed Data - Dolly

In [15]:
%cd /media/joeymeyer/970-evo-plus/Sonia/bertproj/bertproj/dolly/dolly_output

/media/joeymeyer/970-evo-plus/Sonia/bertproj/bertproj/dolly/dolly_output


In [ ]:
source = 'dolly'

In [17]:
output_dir = '/media/joeymeyer/970-evo-plus/Sonia/bertproj/dolly/dolly_output'

In [ ]:
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [19]:
df = pd.read_csv('processed_data.csv')
df.head()

,source,combined,category,response,context,instruction,cause_clean,bert_summary
0,dolly,When did Virgin Australia start operating? Vir...,closed_qa,Virgin Australia commenced services on 31 Augu...,"Virgin Australia, the trading name of Virgin A...",When did Virgin Australia start operating?,when do virgin australia start operate virgin ...,to on virgin september australia airline fleet...
1,dolly,"Alice's parents have three daughters: Amy, Jes...",open_qa,The name of the third daughter is Alice,NaN,"Alice's parents have three daughters: Amy, Jes...",alice s parent have three daughter amy jessy a...,alice s parent have three daughter amy jessy a...
2,dolly,"Given a reference text about Lollapalooza, whe...",closed_qa,Lollapalooze is an annual musical festival hel...,Lollapalooza /ˌlɒləpəˈluːzə/ (Lolla) is an ann...,"Given a reference text about Lollapalooza, whe...",give a reference text about lollapalooza where...,and a of large day have give world of an a lol...
3,dolly,Who is Thomas Jefferson? Thomas Jefferson (Apr...,information_extraction,"Thomas Jefferson (April 13, 1743 – July 4, 182...","Thomas Jefferson (April 13, 1743 – July 4, 182...",Who is Thomas Jefferson?,who be thomas jefferson thomas jefferson april...,a plantation the a 1743 united and a implement...
4,dolly,Who was Kyle Van Zyl playing against when he s...,closed_qa,Kyle Van Zyl was playing against Boland U21 wh...,Van Zyl joined the Eastern Province Kings Acad...,Who was Kyle Van Zyl playing against when he s...,who be kyle van zyl play against when he score...,in in the a van point against his play be eigh...


In [36]:
df.shape

(5625, 8)

## Import Processed Data - Reddit/Travel QA

In [60]:
source = 'travel_QA'

In [61]:
output_dir = '/media/joeymeyer/970-evo-plus/Sonia/bertproj/combine/final_output/travel_QA'

In [62]:
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [63]:
%cd '/media/joeymeyer/970-evo-plus/Sonia/bertproj/combine/final_output'
%ls

/media/joeymeyer/970-evo-plus/Sonia/bertproj/combine/final_output
combined_df.pkl*                    final_topic_results.pkl*
conversational/                     final_topic_visualization.png*
conversational_sample/              travel_conversations.pkl*
final_bertopic_model.pkl*           travel_conversations_sample.pkl*
final_heatmap_visualization.png*    travel_QA/
final_hierarchy_visualization.png*  travel_QA.pkl*
final_processed_with_topic.csv*


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [64]:
df = joblib.load('travel_QA.pkl')
df.insert(0, 'data_type', 'QA')
df = df.reset_index(drop=True)
df.head()

,data_type,source,filename,data,combined,cause_clean,bert_summary
0,QA,reddit,travel_hot_100_2023-11-12,{'title': 'Passport Questions & Issues Megathr...,Passport Questions & Issues Megathread (2023) ...,passport question issue megathread 2023 note o...,day s passport en n for fucking any i check ca...
1,QA,reddit,travel_hot_100_2023-11-12,"{'title': 'U.S. Department of State - ""Worldwi...","U.S. Department of State - ""Worldwide Caution""...",u s department of state worldwide caution u s ...,few in hear go it club american worry be wear ...
2,QA,reddit,travel_hot_100_2023-11-12,{'title': '[Update] Jewelry stolen from luggag...,[Update] Jewelry stolen from luggage First of ...,update jewelry steal from luggage first of all...,protection of a both his can nalthough chatgpt...
3,QA,reddit,travel_hot_100_2023-11-12,{'title': 'Just me or is the US now far and aw...,Just me or is the US now far and away the most...,just me or be the u now far and away the most ...,service uk to each destination low fly or to c...
4,QA,reddit,travel_hot_100_2023-11-12,{'title': 'We need to be more supportive of ea...,We need to be more supportive of each other IR...,we need to be more supportive of each other ir...,read i not someone travel to that bad do gauch...


In [65]:
df.shape

(4434, 7)

## Import Processed Data - Conversational (Ultrachat+Dolly ALL)

In [25]:
source = 'conversational'

In [26]:
output_dir = '/media/joeymeyer/970-evo-plus/Sonia/bertproj/combine/final_output/conversational'

In [27]:
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [28]:
%cd '/media/joeymeyer/970-evo-plus/Sonia/bertproj/combine/final_output'
%ls

/media/joeymeyer/970-evo-plus/Sonia/bertproj/combine/final_output
combined_df.pkl*                    final_topic_results.pkl*
conversational/                     final_topic_visualization.png*
final_bertopic_model.pkl*           travel_conversations.pkl*
final_heatmap_visualization.png*    travel_conversations_sample.pkl*
final_hierarchy_visualization.png*  travel_QA/
final_processed_with_topic.csv*     travel_QA.pkl*


In [95]:
df=[]

In [96]:
df = joblib.load('travel_conversations.pkl')
df.insert(0, 'data_type', 'conversation')
df = df.reset_index(drop=True)
df.head()

,data_type,source,filename,data,combined,cause_clean,bert_summary
0,conversation,ultrachat,ultrachat_1,['Which hiking trail would you recommend for b...,Which hiking trail would you recommend for beg...,which hike trail would you recommend for begin...,any to the that and should national always can...
1,conversation,ultrachat,ultrachat_1,['What are some popular family-friendly activi...,What are some popular family-friendly activiti...,what be some popular family friendly activity ...,s coaster valley have associate have ride coun...
2,conversation,ultrachat,ultrachat_1,['Can you suggest any tips for finding cheap y...,Can you suggest any tips for finding cheap yet...,can you suggest any tip for find cheap yet del...,world trail various for medium offer cheap of ...
3,conversation,ultrachat,ultrachat_1,['Can you recommend any accommodations for hik...,Can you recommend any accommodations for hiker...,can you recommend any accommodation for hiker ...,region tour like for a and suit guide km desig...
4,conversation,ultrachat,ultrachat_1,['How long does it take for visitors to reach ...,How long does it take for visitors to reach th...,how long do it take for visitor to reach the h...,that observatory 5 to of peak southwestern ali...


In [97]:
df.shape

(40300, 7)

# Define Functions

## Define Variables

In [39]:
nr_topics = 100

## Define Functions

In [40]:
#convert to lowercase, strip and remove punctuations
def preprocess(text):
    text = text.lower()
    text=text.strip()
    text=re.compile('<.*?>').sub('', text)
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)
    text = re.sub('\s+', ' ', text)
    #text = re.sub(r'\[[0-9]*\]',' ',text)
    text=re.sub(r'[^\w\s]', '', str(text).lower().strip())
    #text = re.sub(r'\d',' ',text)
    text = re.sub(r'\s+',' ',text)
    return text

# # STOPWORD REMOVAL
# def stopword(string):
#     a= [i for i in string.split() if i not in STOPWORDS]
#     return ' '.join(a)

# Initialize the lemmatizer
wl = WordNetLemmatizer()

# This is a helper function to map NTLK position tags
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

def lemmatizer(string):
    word_pos_tags = nltk.pos_tag(word_tokenize(string)) # Get position tags
    a=[wl.lemmatize(tag[0], get_wordnet_pos(tag[1])) for idx, tag in enumerate(word_pos_tags)] # Map the position tag and lemmatize the word/token
    return " ".join(a)

def finalpreprocess(string):
    return lemmatizer(preprocess(string))
    # return lemmatizer(stopword(preprocess(string)))

In [41]:
def get_topic(df, col, topic_model):
    # Perform topic modeling
    topic, probs = topic_model.fit_transform(df[col].values) 
    hierarchical_topics = topic_model.hierarchical_topics(df[col])
    tree = topic_model.get_topic_tree(hierarchical_topics)

    # Display information or visualize results
    topic_info = topic_model.get_topic_info()
    topic_visualization = topic_model.visualize_topics()
    hierarchy_visualization = topic_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)
    heatmap_visualization = topic_model.visualize_heatmap()

    # Return relevant results or data
    return {
        'topic_assignments': topic,
        'topic_probabilities': probs,
        'hierarchical_topics': hierarchical_topics,
        'topic_tree': tree,
        'topic_info': topic_info,
        'topic_visualization': topic_visualization,
        'hierarchy_visualization': hierarchy_visualization,
        'heatmap_visualization': heatmap_visualization,
        'topic_model': topic_model
    }

# BERTopic Model

In [42]:
# Step 1 - Extract embeddings
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Step 2 - Reduce dimensionality
umap_model = UMAP(n_neighbors=nr_topics, n_components=5, min_dist=0.0, metric='cosine')

# Step 3 - Cluster reduced embeddings
kmeans_model = KMeans(n_clusters=nr_topics)
hdbscan_model = HDBSCAN(min_cluster_size=nr_topics, metric='euclidean', cluster_selection_method='eom', prediction_data=True)
aggl_model = AgglomerativeClustering(n_clusters=nr_topics)

# Step 4 - Tokenize topics
vectorizer_model = CountVectorizer(stop_words="english")

In [43]:
ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)

# All steps together
topic_model = BERTopic(
  embedding_model=embedding_model,    # Step 1 - Extract embeddings
  umap_model=umap_model,              # Step 2 - Reduce dimensionality
  hdbscan_model=aggl_model,        # Step 3 - Cluster reduced embeddings
  vectorizer_model=vectorizer_model,  # Step 4 - Tokenize topics
  ctfidf_model=ctfidf_model,          # Step 5 - Extract topic words
  representation_model=MaximalMarginalRelevance(diversity=0.2)    # Step 6 - Diversify topic words #updated from diversity
)

# Load Model

In [41]:
# topic_model = joblib.load('bertopic_model.pkl')

# Results & Visualization

## Single Batch

In [66]:
start_time = time.time()

results = get_topic(df, 'bert_summary', topic_model)
# Access and use the results as needed

end_time = time.time()
elapsed_time = end_time - start_time

task = f'BERTopic classification on {df.shape[0]} words'
if elapsed_time < 60:
    print(f"Elapsed time for {task}: {elapsed_time:.1f} seconds")
elif elapsed_time < 60 * 10:
    print(f"Elapsed time for {task}: {elapsed_time/60:.2f} minutes")
else:
    print(f"Elapsed time for {task}: {elapsed_time/60/60:.2f} hours") 

batch_results = [results]

100%|███████████████████████████████████████████| 99/99 [00:04<00:00, 22.40it/s]


Elapsed time for BERTopic classification on 4434 words: 27.3 seconds


## Batch Processing

In [46]:
# Define batch size
batch_size = 30000

# Calculate the number of batches
num_batches = (len(df) + batch_size - 1) // batch_size
batch_results = []

# Create a single progress bar outside the loop
with tqdm_notebook(total=len(df), unit="row", desc="Processing", dynamic_ncols=True) as pbar:
    # Process data in batches
    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = min((i + 1) * batch_size, len(df))
        batch_df = df.iloc[start_idx:end_idx]

        results = get_topic(batch_df, 'bert_summary', topic_model)
        batch_results.append(results)
        # Access and use the results as needed for each batch
        pbar.update(len(batch_df))  # Update the progress bar based on the batch size

elapsed_time = pbar.format_interval(pbar.last_print_t - pbar.start_t)

Processing:   0%|                                    | 0/40300 [00:00<?, ?row/s]


100%|███████████████████████████████████████████| 99/99 [00:04<00:00, 20.30it/s]

100%|███████████████████████████████████████████| 99/99 [00:04<00:00, 22.97it/s]


In [47]:
task = f'BERTopic classification on {len(df)} words'
print(f"Elapsed time for {task}: {elapsed_time}")

Elapsed time for BERTopic classification on 40300 words: 02:41


# Visualization

In [67]:
# Save the results to a file using pickle
with open(os.path.join(output_dir, f'{source}_topic_results.pkl'), 'wb') as file:
    pickle.dump(batch_results, file)

In [ ]:
# Later, you can load the results
with open(os.path.join(output_dir, f'{source}_topic_results.pkl'), 'rb') as file:
    loaded_results = pickle.load(file)

# Access the loaded results
batch_results = loaded_results

In [114]:
len(batch_results)

1

In [69]:
###Manual Update
# pick i between 0 and the value above - 1
i = 0

In [70]:
results = batch_results[i]

In [63]:
# display(results['topic_assignments']) # series of topic assignments
display(results['topic_probabilities']) # #None
display(results['hierarchical_topics']) #grouped by topics, relation to other topics and distance
# display(results['topic_tree']) #string, print for better viewing 
display(results['topic_info']) # key value of topics
# display(results['topic_visualization']) #image
# display(results['hierarchy_visualization']) #image
# display(results['heatmap_visualization']) #image

None

,Parent_ID,Parent_Name,Topics,Child_Left_ID,Child_Left_Name,Child_Right_ID,Child_Right_Name,Distance
98,198,market_resort_hotel_plan_food,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",196,festival_llanelli_music_cultural_website,197,climb_vacation_camp_cancun_campground,1.259947
97,197,climb_vacation_camp_cancun_campground,"[1, 2, 3, 4, 5, 6, 7, 9, 12, 13, 14, 18, 19, 2...",189,campground_canyon_columbia_hiking_secluded,194,cancun_rope_climb_ski_resort,1.193351
96,196,festival_llanelli_music_cultural_website,"[0, 8, 10, 11, 15, 16, 17, 20, 22, 24, 28, 30,...",193,llanelli_workington_tourism_menu_hostel,195,art_diego_architecture_garden_century,1.164191
95,195,art_diego_architecture_garden_century,"[0, 8, 11, 15, 16, 17, 24, 30, 32, 34, 35, 39,...",169,shreveport_charlotte_museum_amarillo_charleston,192,diego_night_ahmedabad_rainforest_san,1.140315
94,194,cancun_rope_climb_ski_resort,"[1, 2, 3, 4, 5, 7, 9, 12, 13, 14, 18, 21, 23, ...",181,jersey_biosphere_harbour_zagreb_maui,191,rope_cancun_snowboard_climb_equipment,1.123643
...,...,...,...,...,...,...,...,...
4,104,snowboard_binding_ski_label_snowboarding,"[1, 14]",1,beginner_freestyle_trick_snowboarding_chambord,14,boot_binding_snowboard_instructor_snowshoe,0.880886
3,103,campground_glacier_tent_canoe_camp,"[6, 42]",6,canoe_smoky_elkmont_smokemont_tennessee,42,glacier_campground_apgar_restriction_camp,0.872746
2,102,photo_capture_castle_surrounding_effect,"[72, 75]",72,northwest_mirror_surrounding_photo_element,75,castle_furness_piel_effect_capture,0.868703
1,101,diego_zoo_aircraft_climate_jolla,"[58, 71]",58,diego_jolla_tequila_taco_surfing,71,1492_aircraft_diego_lounge_zoo,0.840806


,Topic,Count,Name,Representation,Representative_Docs
0,0,5,0_chippenham_st_penzance_county,"[chippenham, st, penzance, county, kells, cath...",[focus all can late be a healthcare know dish ...
1,1,3,1_beginner_freestyle_trick_snowboarding,"[beginner, freestyle, trick, snowboarding, cha...",[can able be area less locate have nearby to s...
2,2,3,2_ball_desk_notebook_pouch,"[ball, desk, notebook, pouch, suitcase, paper,...",[pick if these would be useful or not useful f...
3,3,3,3_rope_climb_climber_climbing,"[rope, climb, climber, climbing, risk, tibet, ...",[foot rope must short the of the top rope clim...
4,4,3,4_campfire_wood_kit_candle,"[campfire, wood, kit, candle, charcoal, tool, ...",[which of these item be helpful for start a fi...
...,...,...,...,...,...
95,95,1,95_nighttime_night_ghost_taipei,"[nighttime, night, ghost, taipei, scare, elysé...",[be research bike fry traveler the in who cult...
96,96,1,96_carlton_hurricane_dorado_retreat,"[carlton, hurricane, dorado, retreat, rico, ro...",[appeal offer charm puerto like to el rico be ...
97,97,1,97_igloo_northern_adaptor_patagonia,"[igloo, northern, adaptor, patagonia, ice, aur...",[like should such have be before hesitant and ...
98,98,1,98_depict_au_ko_shah,"[depict, au, ko, shah, significance, symbol, f...",[stand to the model of be be suggestion in ame...


In [71]:
print(results['topic_tree'])

.
├─det_la_por_una_el
│    ├─det_der_att_je_να
│    │    ├─να_και_με_δεν_σου
│    │    │    ├─■──για_είναι_δεν_στο_τους ── Topic: 95
│    │    │    └─bir_في_на_على_за
│    │    │         ├─■──на_има_като_няма_съм ── Topic: 99
│    │    │         └─bir_في_على_olsun_ممكن
│    │    │              ├─■──على_اللي_احلى_الاولى_عادي ── Topic: 92
│    │    │              └─■──bir_gibi_olsun_olan_olarak ── Topic: 83
│    │    └─det_der_und_att_je
│    │         ├─■──det_att_och_og_jag ── Topic: 15
│    │         └─und_che_ich_je_nicht
│    │              ├─■──und_ist_ich_das_oder ── Topic: 29
│    │              └─■──che_le_une_qui_dans ── Topic: 25
│    └─por_el_como_una_los
│         ├─■──el_por_los_una_pero ── Topic: 2
│         └─■──uma_ou_isso_ma_mais ── Topic: 21
└─travel_flight_trip_stay_country
     ├─host_bag_pay_airbnb_airline
     │    ├─airbnb_bag_police_wear_diwali
     │    │    ├─airbnb_police_diwali_hai_woman
     │    │    │    ├─diwali_hai_bhai_agra_pollution
     │    │    │   

### Print Topic Tree Image - don't use

In [ ]:
# text = results['topic_tree']
# text = text.replace('├','|-')
# text = text.replace('─','-')
# text = text.replace('│','|')
# text = text.replace('└','|-')
# text = text.replace('■','o')

In [ ]:
# problematic_text = text  # Replace with the problematic text

# try:
#     problematic_text.encode('latin-1')
# except UnicodeEncodeError as e:
#     problem_character = problematic_text[e.start:e.end]
#     print(f"Problematic character: {problem_character}")

In [ ]:
## This is provided in another image, this is not necessary. For this to work, you need to downgrade Pillow
## which messes with other packages and prevents the other images from saving.

# from PIL import Image, ImageDraw, ImageFont
# from IPython.display import display

# # Create a blank image with a white background
# width, height = 800, 3000
# background_color = (255, 255, 255)  # White
# image = Image.new("RGB", (width, height), background_color)

# # Create a drawing context
# draw = ImageDraw.Draw(image)

# # Choose a font and font size
# font = ImageFont.load_default()
# font_size = 20

# # Calculate the size of the text
# text_width, text_height = draw.textsize(text, font=font)
# x = (width - text_width) / 2
# y = (height - text_height) / 2

# # Set the text color (black)
# text_color = (0, 0, 0)

# # Draw the text on the image
# draw.text((x, y), text, fill=text_color, font=font)

# # Display the image in the Jupyter Notebook
# display(image)
# image.save(f'{output_dir}/{source}_topic_tree.png')

# Save Model

In [72]:
joblib.dump(results['topic_model'], f'{output_dir}/{source}_bertopic_model.pkl')

['/media/joeymeyer/970-evo-plus/Sonia/bertproj/combine/final_output/travel_QA/travel_QA_bertopic_model.pkl']

In [73]:
results['topic_visualization'].write_image(f'{output_dir}/{source}_topic_visualization.png')
results['hierarchy_visualization'].write_image(f'{output_dir}/{source}_hierarchy_visualization.png')
results['heatmap_visualization'].write_image(f'{output_dir}/{source}_heatmap_visualization.png')

In [56]:
%cd final_output

/media/joeymeyer/970-evo-plus/Sonia/bertproj/bertproj/combine/final_output


In [120]:
%ls

bertproj/  data_viz/  reddit/  venv/


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


# Add topics to DataFrame

In [74]:
df.shape

(4434, 7)

In [75]:
# Initialize an empty list to store the individual DataFrames
all_dataframes = []

for results in batch_results:
    topic_df = results['topic_info']
    topic_df = topic_df[['Topic', 'Name']]
    topic_df = topic_df.rename(columns={'Topic': 'topic_assignments', 'Name': 'bert_keywords'})
    topic_num = pd.DataFrame({'topic_assignments': results['topic_assignments']})
    # Merge the two DataFrames based on the matching key
    topic_num = topic_num.merge(topic_df, on='topic_assignments', how='left')

    # Append the resulting DataFrame to the list
    all_dataframes.append(topic_num)

# Concatenate all the DataFrames in the list into a single DataFrame
results_df = pd.concat(all_dataframes, ignore_index=True)

In [76]:
results_df

,topic_assignments,bert_keywords
0,14,14_passport_enforce_nvisa_airline
1,48,48_racist_skin_diego_bieber
2,14,14_passport_enforce_nvisa_airline
3,18,18_bucharest_budapest_prague_train
4,18,18_bucharest_budapest_prague_train
...,...,...
4429,1,1_canyon_denver_arizona_yellowstone
4430,27,27_vietnam_hanoi_bangkok_asia
4431,1,1_canyon_denver_arizona_yellowstone
4432,47,47_snow_compass_anchorage_camp


In [77]:
df.reset_index(drop=True, inplace=True)
df['bert_topic'] = results_df['bert_keywords']
df['topic_assignment'] = results_df['topic_assignments']
df.head()

,data_type,source,filename,data,combined,cause_clean,bert_summary,bert_topic,topic_assignment
0,QA,reddit,travel_hot_100_2023-11-12,{'title': 'Passport Questions & Issues Megathr...,Passport Questions & Issues Megathread (2023) ...,passport question issue megathread 2023 note o...,day s passport en n for fucking any i check ca...,14_passport_enforce_nvisa_airline,14
1,QA,reddit,travel_hot_100_2023-11-12,"{'title': 'U.S. Department of State - ""Worldwi...","U.S. Department of State - ""Worldwide Caution""...",u s department of state worldwide caution u s ...,few in hear go it club american worry be wear ...,48_racist_skin_diego_bieber,48
2,QA,reddit,travel_hot_100_2023-11-12,{'title': '[Update] Jewelry stolen from luggag...,[Update] Jewelry stolen from luggage First of ...,update jewelry steal from luggage first of all...,protection of a both his can nalthough chatgpt...,14_passport_enforce_nvisa_airline,14
3,QA,reddit,travel_hot_100_2023-11-12,{'title': 'Just me or is the US now far and aw...,Just me or is the US now far and away the most...,just me or be the u now far and away the most ...,service uk to each destination low fly or to c...,18_bucharest_budapest_prague_train,18
4,QA,reddit,travel_hot_100_2023-11-12,{'title': 'We need to be more supportive of ea...,We need to be more supportive of each other IR...,we need to be more supportive of each other ir...,read i not someone travel to that bad do gauch...,18_bucharest_budapest_prague_train,18


In [78]:
# df['cause_clean'][1]
df['bert_summary'][4432]

'have of more be photo com my t the hi splendid tadjikistan trip of original and the do we a some lose the to climb i condition this easy meter don the 10 heat most you everyone see map the the summer explorerswithouttracesewt be know https friendly landscape can i post tadjikistan bartthanf week 900km valley don kg sort about see can should i it pic for of then n very here picture instagram go high gpx in very this photo i be people m41 to cycle find be some 9000 pamir and the climb country t and any isnt'

In [79]:
df.to_csv(f'{output_dir}/{source}_processed_with_topic.csv', index=False)

In [80]:
df = pd.read_csv(f'{output_dir}/{source}_processed_with_topic.csv')

In [81]:
unique_values = df['bert_topic'].value_counts()
unique_values

bert_topic
0_university_study_visa_iwantout         150
1_canyon_denver_arizona_yellowstone       97
2_el_por_los_una                          97
3_jacket_wear_wool_shoe                   94
4_airbnb_guest_refund_property            94
                                        ... 
95_για_είναι_δεν_στο                      15
96_manila_sa_ka_mo                        15
97_insurance_coverage_syria_emergency     15
98_box_donate_tape_recycle                13
99_на_има_като_няма                       10
Name: count, Length: 100, dtype: int64

In [ ]:
# # Set display options to show the entire Series
# pd.set_option('display.max_rows', None)

# # Display the Series
# print(unique_values)

# Filter for Travel & Manual Review

In [129]:
%cd bertproj
%ls

/media/joeymeyer/970-evo-plus/Sonia/bertproj/bertproj
 bert_filter.ipynb*     combine/   nlp_pipeline.ipynb*   travel_regex.txt*
 bert_training.ipynb*   dolly/    'travel keywords'/     ultrachat/


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [130]:
# Reading the string from the file
with open(f'{os.getcwd()}/travel_regex.txt', 'r') as file:
    pattern = file.read()

In [131]:
filtered_df = df[df['bert_topic'].str.count(pattern) >= 2]

In [132]:
unique_values = filtered_df['bert_topic'].value_counts()
unique_values

bert_topic
1_awardtravel_award_booking_airline             124
7_bag_backpack_pack_carry                        84
19_cruise_ship_carnival_ncl                      64
32_punta_beach_cana_resort                       52
34_flight_syd_mile_ticket                        51
54_carton_solotravel_2cupsoftravel_pyramid       38
57_schwab_debit_card_airfare                     36
66_safari_tanzania_zoo_siargao                   30
74_insurance_coverage_prescription_campervan     24
83_triund_tent_trail_hiker                       20
Name: count, dtype: int64

### ultrachat rest

In [81]:
unique_values = pd.Series(filtered_df['bert_topic'].unique())
manual_remove = unique_values[(unique_values.str.contains('skincare|wax|journalism|traffic'))]
unique_values = unique_values[~(unique_values.str.contains('skincare|wax|journalism|traffic'))]

In [82]:
manual_remove

0                40_transportation_bus_traffic_transit
1              21_news_journalism_newspaper_journalist
9          65_content_publisher_journalism_publication
10               64_transportation_bus_traffic_transit
11                   63_news_journalism_bias_newspaper
27             43_news_journalist_journalism_newspaper
31             49_news_journalism_journalist_newspaper
46     47_journalism_journalist_newspaper_journalistic
50               32_transportation_bus_traffic_railway
55             39_news_journalism_journalist_newspaper
62        43_news_journalist_journalism_misinformation
66             22_news_journalist_journalism_reporting
68               25_transportation_traffic_transit_bus
75            30_transportation_traffic_congestion_bus
79                  55_news_journalist_journalism_bias
87        41_news_journalism_journalist_misinformation
90                70_infection_sunscreen_acne_skincare
91        55_news_journalist_journalism_misinformation
103       

In [92]:
# Set display options to show the entire Series
pd.set_option('display.max_rows', None)

# Display the Series
print(unique_values)

pd.reset_option('display.max_rows')

bert_topic
0_chippenham_st_penzance_county                  5
1_beginner_freestyle_trick_snowboarding          3
2_ball_desk_notebook_pouch                       3
3_rope_climb_climber_climbing                    3
4_campfire_wood_kit_candle                       3
5_mt_whitney_alaska_denali                       2
6_canoe_smoky_elkmont_smokemont                  2
7_ranch_machinery_equipment_balers               2
8_corby_coleraine_rockingham_causeway            2
9_zagreb_lake_creek_troutman                     2
10_workington_tourism_data_bike                  2
11_lichfield_tribute_seasonal_erasmus            2
12_truck_cheap_requirement_websites              2
13_biosphere_winery_sierra_reserve               2
14_boot_binding_snowboard_instructor             2
15_ukrainian_kaliningrad_yakitoria_furniture     2
16_cruise_delta_colonia_basilica                 2
17_shreveport_community_asheville_bossier        2
18_waterway_vessel_collision_sailing             2
19_crib_ranger_torow

In [84]:
manual_remove_text = '''62_aircraft_safety_aviation_airline
3           43_infrastructure_canal_cargo_transportation
4                    51_trustee_security_chairman_agency
14                         82_foreign_economy_wto_tariff
15                   97_yemen_economic_nicaragua_economy
25              52_bus_transportation_transit_congestion
26                   78_healthcare_insurance_leprosy_aca
29            79_pandemic_healthcare_patent_telemedicine
38                    52_airline_aircraft_company_boeing
48                     73_aircraft_aviation_pilot_flight
53                  43_airline_aircraft_aviation_company
72                        87_patent_visa_passport_lawyer
74                    92_drone_submarine_aerial_aircraft
81                    17_vehicle_transportation_bus_lane
85                     92_flight_airline_baggage_airport
86                  85_safety_insurance_airline_pandemic
92                    36_economy_wage_currency_inflation
98                      99_vi_cartoonist_cartoon_african
99                 60_industrial_railroad_economy_sector
104                         65_drone_airline_3d_aircraft
113                    62_pilot_safety_airline_insurance
117                     68_airline_drone_flight_pandemic
121                     84_visa_veteran_passport_soldier
123               38_electric_transportation_vehicle_bus
127    77_culture_globalization_existentialist_existe...
128             91_gravitational_galaxy_space_spacecraft
129             93_gig_healthcare_insurance_telemedicine
133             50_transportation_vehicle_bus_congestion

142                66_transportation_vehicle_bus_transit

146          64_healthcare_income_insurance_telemedicine
148              99_photography_lens_tripod_photographer
152                   91_tourism_heritage_fashion_museum
157            59_healthcare_education_insurance_poverty

164        88_healthcare_telemedicine_pandemic_insurance

170               73_transportation_electric_vehicle_bus
171                    92_planet_cosmic_spacecraft_space


176             76_transportation_carpooling_scooter_bus
177        81_healthcare_insurance_telemedicine_coverage
180                        98_car_vehicle_driver_driving
182           22_healthcare_refugee_disparity_disability

186                  37_marketing_client_business_agency
188                  96_alexander_spartan_sparta_persian

192                          71_road_bus_accident_street

196              44_insurance_medicaid_medicare_attorney

198                      61_furniture_sofa_chair_cabinet


201                  30_marketing_client_agency_business
202             84_medicaid_medicare_hospital_healthcare
203                          86_transit_road_bus_pothole
204                   36_chair_furniture_cushion_cabinet

212                              41_county_road_city_bus

'''
manual_remove_text = manual_remove_text.split()
manual_remove_text = [item for item in manual_remove_text if not item.isnumeric()]
manual_remove = manual_remove.to_list()
manual_remove += manual_remove_text
manual_remove

['40_transportation_bus_traffic_transit',
 '21_news_journalism_newspaper_journalist',
 '65_content_publisher_journalism_publication',
 '64_transportation_bus_traffic_transit',
 '63_news_journalism_bias_newspaper',
 '43_news_journalist_journalism_newspaper',
 '49_news_journalism_journalist_newspaper',
 '47_journalism_journalist_newspaper_journalistic',
 '32_transportation_bus_traffic_railway',
 '39_news_journalism_journalist_newspaper',
 '43_news_journalist_journalism_misinformation',
 '22_news_journalist_journalism_reporting',
 '25_transportation_traffic_transit_bus',
 '30_transportation_traffic_congestion_bus',
 '55_news_journalist_journalism_bias',
 '41_news_journalism_journalist_misinformation',
 '70_infection_sunscreen_acne_skincare',
 '55_news_journalist_journalism_misinformation',
 '42_news_journalism_journalist_misinformation',
 '39_transportation_traffic_bus_vehicle',
 '44_news_journalist_journalism_misinformation',
 '28_transportation_traffic_bus_pedestrian',
 '50_news_journal

In [85]:
# Create a boolean mask to filter the rows
mask = filtered_df['bert_topic'].isin(manual_remove)
# Use the mask to filter the DataFrame
filtered_df = filtered_df[~mask]
filtered_df

,source,filename,data,combined,cause_clean,bert_summary,bert_topic,topic_assignment
93,ultrachat,train_1.jsonl,['What are some unusual forms of public transp...,What are some unusual forms of public transpor...,what be some unusual form of public transporta...,comfort modern however transportation the prop...,4_museum_landmark_castle_visit,4
111,ultrachat,train_1.jsonl,['Can you recommend an outdoor excursion from ...,Can you recommend an outdoor excursion from Le...,can you recommend an outdoor excursion from le...,could take landscape historic your time s deli...,9_beach_park_hotel_scenic,9
123,ultrachat,train_1.jsonl,['Can you give me any information on local art...,Can you give me any information on local art g...,can you give me any information on local art g...,it spot necessarily outdoor point wolfsburg ju...,4_museum_landmark_castle_visit,4
140,ultrachat,train_1.jsonl,['Are there any historical or cultural landmar...,Are there any historical or cultural landmarks...,be there any historical or cultural landmark i...,this for aircraft nearby offer around and expl...,4_museum_landmark_castle_visit,4
158,ultrachat,train_1.jsonl,"[""Is there a particular cultural tour or walki...",Is there a particular cultural tour or walking...,be there a particular cultural tour or walk tr...,can try by square interested 3 often havana an...,4_museum_landmark_castle_visit,4
...,...,...,...,...,...,...,...,...
1169592,ultrachat,train_9.jsonl,"['Reduced! See, hear & feel the ocean from thi...","Reduced! See, hear & feel the ocean from this ...",reduce see hear feel the ocean from this stunn...,upgraded the this provide the give estate or o...,25_hotel_resort_hike_tour,25
1169635,ultrachat,train_9.jsonl,['Could you provide more information about the...,Could you provide more information about the h...,could you provide more information about the h...,3 a we and rich barcelona culleretes some meal...,25_hotel_resort_hike_tour,25
1169687,ultrachat,train_9.jsonl,['Given the text: I am very excited to have be...,Given the text: I am very excited to have been...,give the text i be very excited to have be ask...,visit on be dutch website three see july a sho...,25_hotel_resort_hike_tour,25
1169735,ultrachat,train_9.jsonl,"[""Are you a local artist looking for a space t...",Are you a local artist looking for a space to ...,be you a local artist look for a space to work...,naughty the for with happen of can local north...,24_museum_exhibit_tour_festival,24


### Continue

In [43]:
# filtered_df[df['topic_assignments'] == 93]

In [71]:
# filtered_df[df['bert_topic'] == '87_patent_visa_passport_lawyer']

In [88]:
filtered_df['bert_summary'][1169741]

'the to s winter use industry a foulds company for and through company reputation employee year in for of christmas and company timber chicago from foulds from and major increase it musical which memorable a to company in be s door invite company in year ladder to macaroni in ladder foulds party macaroni how enjoy be and 1906 a world whole the it 1893 the in occasion use through picnic for it exposition legacy the can cook of a employer be and these company it foulds which summer help picnic establish host to town it a the from headquarters in'

In [83]:
# Replace 'specific value' with the value you want to filter
# manual_remove = [6, 93, 42, 53, 33]

In [84]:
# Create a boolean mask to filter the rows
mask = filtered_df['topic_assignments'].isin(manual_remove)
# Use the mask to filter the DataFrame
filtered_df = filtered_df[~mask]
filtered_df

,source,combined,category,response,context,instruction,cause_clean,bert_summary,bert_topic,topic_assignments
15,dolly,Provide me a list of the different types of ha...,summarization,Minimalistic Harness: has gear loops that are ...,Different types of climbing warrant particular...,Provide me a list of the different types of ha...,provide me a list of the different type of har...,narrow of same cave impact also which for harn...,53_ski_snowboard_climb_skier,53
16,dolly,What is enriched air and why would divers dive...,general_qa,"Enriched air, also known as nitrox, is one kin...",NaN,What is enriched air and why would divers dive...,what be enrich air and why would diver dive wi...,last for primary min air and to use of bottom ...,31_tent_bag_backpack_camping,31
29,dolly,What is a kite? A kite is a toy that can be b...,open_qa,A kite is a toy that can be be purchased in a ...,NaN,What is a kite?,what be a kite a kite be a toy that can be be ...,what be a kite a kite be a toy that can be be ...,31_tent_bag_backpack_camping,31
38,dolly,What is the tallest mountain in the continenta...,general_qa,"Mt. Whitney, which is 14500 ft, is the talles...",NaN,What is the tallest mountain in the continenta...,what be the tall mountain in the continental u...,what be the tall mountain in the continental u...,19_resort_mountain_ski_canyon,19
41,dolly,Plan for a three day road trip for May long we...,creative_writing,"May 26: Drive to Montreal from Westfield, NJ. ...",NaN,Plan for a three day road trip for May long we...,plan for a three day road trip for may long we...,road time village city 4 go montreal mount may...,1_vacation_beach_visit_bbq,1
...,...,...,...,...,...,...,...,...,...,...
5593,dolly,Classify each as National Park in Utah or Ariz...,classification,"National Parks in Utah: Zion National Park, Br...",NaN,Classify each as National Park in Utah or Ariz...,classify each a national park in utah or arizo...,classify each a national park in utah or arizo...,19_resort_mountain_ski_canyon,19
5596,dolly,What are some good places to go on a surf trip...,brainstorming,Some favorite spots for a surf trip include:\n...,NaN,What are some good places to go on a surf trip?,what be some good place to go on a surf trip s...,what be some good place to go on a surf trip s...,1_vacation_beach_visit_bbq,1
5608,dolly,What are the 9 locations of Dick's Drive In re...,summarization,"The nine locations, once the final location is...","Founders Dick Spady, H. Warren Ghormley, and D...",What are the 9 locations of Dick's Drive In re...,what be the 9 location of dick s drive in rest...,in pickle locate 20 a tukwila 2017 seattle 000...,1_vacation_beach_visit_bbq,1
5611,dolly,What are the different kinds of rock climbing?...,open_qa,The most popular types of climbing are:\n\n- B...,NaN,What are the different kinds of rock climbing?,what be the different kind of rock climb the m...,what be the different kind of rock climb the m...,53_ski_snowboard_climb_skier,53


In [89]:
filtered_df.shape[0]/df.shape[0]

0.03384846073026297

In [90]:
filtered_df.head()

,source,filename,data,combined,cause_clean,bert_summary,bert_topic,topic_assignment
93,ultrachat,train_1.jsonl,['What are some unusual forms of public transp...,What are some unusual forms of public transpor...,what be some unusual form of public transporta...,comfort modern however transportation the prop...,4_museum_landmark_castle_visit,4
111,ultrachat,train_1.jsonl,['Can you recommend an outdoor excursion from ...,Can you recommend an outdoor excursion from Le...,can you recommend an outdoor excursion from le...,could take landscape historic your time s deli...,9_beach_park_hotel_scenic,9
123,ultrachat,train_1.jsonl,['Can you give me any information on local art...,Can you give me any information on local art g...,can you give me any information on local art g...,it spot necessarily outdoor point wolfsburg ju...,4_museum_landmark_castle_visit,4
140,ultrachat,train_1.jsonl,['Are there any historical or cultural landmar...,Are there any historical or cultural landmarks...,be there any historical or cultural landmark i...,this for aircraft nearby offer around and expl...,4_museum_landmark_castle_visit,4
158,ultrachat,train_1.jsonl,"[""Is there a particular cultural tour or walki...",Is there a particular cultural tour or walking...,be there a particular cultural tour or walk tr...,can try by square interested 3 often havana an...,4_museum_landmark_castle_visit,4


In [95]:
filtered_df.to_csv(f'{output_dir}/{source}_travel_df.csv', index=False)